In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import trim
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import count


import datetime
import numpy as np
import pandas as pd

In [0]:
df = spark.read.format('csv').load("dbfs:/FileStore/shared_uploads/qudus.ayoola123@gmail.com/dec17pub.txt", header = None) 

In [0]:
df2 = df.select(trim(substring(df._c0, 1, 15)).alias('household_identifier'), \
               trim(concat(substring(df._c0, 18, 4), lit("/"), substring(df._c0, 16, 2))).alias('interview_time'), \
               trim(substring(df._c0, 24, 3)).alias('final_survey_outcome'), \
               trim(substring(df._c0, 31, 2)).alias('housing_unit_type'), \
               trim(substring(df._c0, 61, 2)).alias('household_type'), \
               trim(substring(df._c0, 33, 2)).alias('apartment/household_has_telephone'), \
               regexp_replace(trim(substring(df._c0, 35, 2)), "-", "").alias('apartment/household_can_access_telephone_elsewhere'), \
               regexp_replace(trim(substring(df._c0, 37, 2)), "-", "").alias('is_telephone_interview_acceptable'), \
               regexp_replace(trim(substring(df._c0, 65, 2)), "-", "").alias('interview_type'), \
               regexp_replace(trim(substring(df._c0, 39, 2)), "-", "").alias('family_income'), \
               trim(substring(df._c0, 91, 1)).alias('geographical_division/location'), \
               trim(substring(df._c0, 139, 2)).alias('race'))


In [0]:
display(df2.head(10))

household_identifier,interview_time,final_survey_outcome,housing_unit_type,household_type,apartment/household_has_telephone,apartment/household_can_access_telephone_elsewhere,is_telephone_interview_acceptable,interview_type,family_income,geographical_division/location,race
000004795110719,2017/12,201,1,1,1,1,1,2,9,6,1
000004795110719,2017/12,201,1,1,1,1,1,2,9,6,1
000071691004941,2017/12,201,1,1,1,1,1,1,11,6,1
000071691004941,2017/12,201,1,1,1,1,1,1,11,6,1
000071691004941,2017/12,201,1,1,1,1,1,1,11,6,1
000110177987986,2017/12,201,1,1,1,1,1,1,14,6,2
000110177987986,2017/12,201,1,1,1,1,1,1,14,6,2
000110206593381,2017/12,213,1,0,-1,1,0,1,1,6,-1
000110284815680,2017/12,201,1,7,1,1,1,1,9,6,1
000110327856469,2017/12,201,1,7,1,1,1,1,5,6,2


In [0]:
df3 = df2.select(df2.household_identifier, \
                df2.interview_time, \
                 
                 when(df2['final_survey_outcome'] == lit("1"), lit("FULLY COMPLETE CATI INTERVIEW")). \
                when(df2['final_survey_outcome'] == lit("2"), lit("PARTIALLY COMPLETED CATI INTERVIEW")). \
                when(df2['final_survey_outcome'] == lit("3"), lit("COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH")). \
                when(df2['final_survey_outcome'] == lit("4"), lit("PARTIAL, NOT COMPLETE AT CLOSEOUT")). \
                when(df2['final_survey_outcome'] == lit("5"), lit("LABOR FORCE COMPLETE, SUPPLEMENT INCOMPLETE - CATI")). \
                when(df2['final_survey_outcome'] == lit("6"), lit("LF COMPLETE, SUPPLEMENT DK ITEMS INCOMPLETE AT CLOSEOUT–ASEC ONLY")). \
                when(df2['final_survey_outcome'] == lit("20"), lit("HH OCCUPIED ENTIRELY BY ARMED FORCES MEMBERS OR ALL UNDER 15 YEARS OF AGE")). \
                when(df2['final_survey_outcome'] == lit("201"), lit("CAPI COMPLETE")). \
                when(df2['final_survey_outcome'] == lit("202"), lit("CALLBACK NEEDED")). \
                when(df2['final_survey_outcome'] == lit("203"), lit("SUFFICIENT PARTIAL - PRECLOSEOUT")). \
                when(df2['final_survey_outcome'] == lit("204"), lit("SUFFICIENT PARTIAL - AT CLOSEOUT")). \
                when(df2['final_survey_outcome'] == lit("205"), lit("LABOR FORCE COMPLETE, - SUPPL. INCOMPLETE - CAPI")). \
                when(df2['final_survey_outcome'] == lit("213"), lit("LANGUAGE BARRIER")). \
                when(df2['final_survey_outcome'] == lit("214"), lit("UNABLE TO LOCATE")). \
                when(df2['final_survey_outcome'] == lit("216"), lit("NO ONE HOME")). \
                when(df2['final_survey_outcome'] == lit("217"), lit("TEMPORARILY ABSENT")). \
                when(df2['final_survey_outcome'] == lit("218"), lit("REFUSED")). \
                when(df2['final_survey_outcome'] == lit("219"), lit("OTHER OCCUPIED - SPECIFY")). \
                when(df2['final_survey_outcome'] == lit("223"), lit("ENTIRE HOUSEHOLD ARMED FORCES")). \
                when(df2['final_survey_outcome'] == lit("224"), lit("ENTIRE HOUSEHOLD UNDER 15")). \
                when(df2['final_survey_outcome'] == lit("225"), lit("TEMP. OCCUPIED W/PERSONS WITH URE")). \
                when(df2['final_survey_outcome'] == lit("226"), lit("VACANT REGULAR")). \
                when(df2['final_survey_outcome'] == lit("227"), lit("VACANT - STORAGE OF HHLD FURNITURE")). \
                when(df2['final_survey_outcome'] == lit("228"), lit("UNFIT, TO BE DEMOLISHED")). \
                when(df2['final_survey_outcome'] == lit("229"), lit("UNDER CONSTRUCTION, NOT READY")). \
                when(df2['final_survey_outcome'] == lit("230"), lit("CONVERTED TO TEMP BUSINESS OR STORAGE")). \
                when(df2['final_survey_outcome'] == lit("231"), lit("UNOCCUPIED TENT OR TRAILER SITE")). \
                when(df2['final_survey_outcome'] == lit("232"), lit("PERMIT GRANTED - CONSTRUCTION NOT STARTED")). \
                when(df2['final_survey_outcome'] == lit("233"), lit("OTHER - SPECIFY")). \
                when(df2['final_survey_outcome'] == lit("240"), lit("DEMOLISHED")). \
                when(df2['final_survey_outcome'] == lit("241"), lit("HOUSE OR TRAILER MOVED")). \
                when(df2['final_survey_outcome'] == lit("242"), lit("OUTSIDE SEGMENT")). \
                when(df2['final_survey_outcome'] == lit("243"), lit("CONVERTED TO PERM. BUSINESS OR STORAGE")). \
                when(df2['final_survey_outcome'] == lit("244"), lit("MERGED")). \
                when(df2['final_survey_outcome'] == lit("245"), lit("CONDEMNED")). \
                when(df2['final_survey_outcome'] == lit("246"), lit("BUILT AFTER APRIL 1, 2000")). \
                when(df2['final_survey_outcome'] == lit("247"), lit("UNUSED SERIAL NO./LISTING SHEET LINE")). \
                when(df2['final_survey_outcome'] == lit("248"), lit("OTHER - SPECIFY")). \
                when(df2['final_survey_outcome'] == lit("256"), lit("REMOVED DURING SUB-SAMPLING")). \
                when(df2['final_survey_outcome'] == lit("257"), lit("UNIT ALREADY HAD A CHANCE OF SELECTION")). \
                 otherwise('UNDEFINED').alias('final_survey_outcome'), \
                when(df2.housing_unit_type == 1, lit("HOUSE, APARTMENT, FLAT")). \
                 when(df2.housing_unit_type == 2, lit("HU IN NONTRANSIENT HOTEL, MOTEL, ETC.")). \
                when(df2.housing_unit_type == 3, lit("HU PERMANENT IN TRANSIENT HOTEL, MOTEL")). \
                when(df2.housing_unit_type == 4, lit("HU IN ROOMING HOUSE")). \
                when(df2.housing_unit_type == 5, lit("MOBILE HOME OR TRAILER W/NO PERM. ROOM ADDED")). \
                when(df2.housing_unit_type == 6, lit("MOBILE HOME OR TRAILER W/1 OR MORE PERM. ROOMS ADDED")). \
                when(df2.housing_unit_type == 7, lit("HU NOT SPECIFIED ABOVE")). \
                when(df2.housing_unit_type == 8, lit("QUARTERS NOT HU IN ROOMING OR BRDING HS")). \
                when(df2.housing_unit_type == 9, lit("UNIT NOT PERM. IN TRANSIENT HOTL, MOTL")). \
                when(df2.housing_unit_type == 10, lit("UNOCCUPIED TENT SITE OR TRLR SITE")). \
                when(df2.housing_unit_type == 11, lit("STUDENT QUARTERS IN COLLEGE DORM")). \
                when(df2.housing_unit_type == 12, lit("OTHER UNIT NOT SPECIFIED ABOVE")). \
                 otherwise('UNDEFINED').alias('housing_unit_type'), \
                when(df2.household_type == 1, lit("HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF)")). \
                when(df2.household_type == 2, lit("HUSB/WIFE PRIM. FAMILY (EITHER/BOTH AF)")). \
                when(df2.household_type == 3, lit("UNMARRIED CIVILIAN MALE-PRIM. FAM HHLDER")). \
                when(df2.household_type == 4, lit("UNMARRIED CIV. FEMALE-PRIM FAM HHLDER")). \
                when(df2.household_type == 5, lit("PRIMARY FAMILY HHLDER-RP IN AF, UNMAR.")). \
                when(df2.household_type == 6, lit("CIVILIAN MALE PRIMARY INDIVIDUAL")). \
                when(df2.household_type == 7, lit("CIVILIAN FEMALE PRIMARY INDIVIDUAL")). \
                when(df2.household_type == 8, lit("PRIMARY INDIVIDUAL HHLD-RP IN AF")). \
                when(df2.household_type == 9, lit("GROUP QUARTERS WITH FAMILY")). \
                when(df2.household_type == 10, lit("GROUP QUARTERS WITHOUT FAMILY")). \
                 otherwise('UNDEFINED').alias('household_type'), \
                when(df2['apartment/household_has_telephone'] == 1, lit("YES")). \
                when(df2['apartment/household_has_telephone'] == 2, lit("NO")). \
                 otherwise('UNDEFINED').alias('apartment/household_has_telephone'), \
                when(df2['apartment/household_can_access_telephone_elsewhere'] == lit("1"), lit("YES")). \
                when(df2['apartment/household_can_access_telephone_elsewhere'] == lit("2"), \
                lit("NO")).alias('apartment/household_can_access_telephone_elsewhere'), \
                when(df2['is_telephone_interview_acceptable'] == lit("1"), lit("YES")). \
                when(df2['is_telephone_interview_acceptable'] == lit("2"), \
                lit("NO")).otherwise('UNDEFINED').alias('is_telephone_interview_acceptable'), \
                when(df2['interview_type'] == lit("0"), lit("NONINTERVIEW/INDETERMINATE")). \
                when(df2['interview_type'] == lit("1"), lit("PERSONAL")). \
                when(df2['interview_type'] == lit("2"), lit("TELEPHONE")).\
                 otherwise('UNDEFINED').alias('interview_type'), \
                when(df2['family_income'] == lit("1"), lit("LESS THAN $5,000")). \
                when(df2['family_income'] == lit("2"), lit("5,000 TO 7,499")). \
                when(df2['family_income'] == lit("3"), lit("7,500 TO 9,999")). \
                when(df2['family_income'] == lit("4"), lit("10,000 TO 12,499")). \
                when(df2['family_income'] == lit("5"), lit("12,500 TO 14,999")). \
                when(df2['family_income'] == lit("6"), lit("15,000 TO 19,999")). \
                when(df2['family_income'] == lit("7"), lit("20,000 TO 24,999")). \
                when(df2['family_income'] == lit("8"), lit("25,000 TO 29,999")). \
                when(df2['family_income'] == lit("9"), lit("30,000 TO 34,999")). \
                when(df2['family_income'] == lit("10"), lit("35,000 TO 39,999")). \
                when(df2['family_income'] == lit("11"), lit("40,000 TO 49,999")). \
                when(df2['family_income'] == lit("12"), lit("50,000 TO 59,999")). \
                when(df2['family_income'] == lit("13"), lit("60,000 TO 74,999")). \
                when(df2['family_income'] == lit("14"), lit("75,000 TO 99,999")). \
                when(df2['family_income'] == lit("15"), lit("100,000 TO 149,999")). \
                when(df2['family_income'] == lit("16"), lit("150,000 OR MORE")). \
                 otherwise('UNDEFINED').alias('family_income'), \
                when(df2['geographical_division/location'] == lit("1"), lit("NEW ENGLAND")). \
                when(df2['geographical_division/location'] == lit("2"), lit("MIDDLE ATLANTIC")). \
                when(df2['geographical_division/location'] == lit("3"), lit("EAST NORTH CENTRAL")). \
                when(df2['geographical_division/location'] == lit("4"), lit("WEST NORTH CENTRAL")). \
                when(df2['geographical_division/location'] == lit("5"), lit("SOUTH ATLANTIC")). \
                when(df2['geographical_division/location'] == lit("6"), lit("EAST SOUTH CENTRAL")). \
                when(df2['geographical_division/location'] == lit("7"), lit("WEST SOUTH CENTRAL")). \
                when(df2['geographical_division/location'] == lit("8"), lit("MOUNTAIN")). \
                when(df2['geographical_division/location'] == lit("9"), lit("PACIFIC")). \
                otherwise('UNDEFINED').alias('geographical_division/location'), \
                when(df2['race'] == lit("1"), lit("White Only")). \
                when(df2['race'] == lit("2"), lit("Black Only")). \
                when(df2['race'] == lit("3"), lit("American Indian, Alaskan Native Only")). \
                when(df2['race'] == lit("4"), lit("Asian Only")). \
                when(df2['race'] == lit("5"), lit("Hawaiian/Pacific Islander Only")). \
                when(df2['race'] == lit("6"), lit("White-Black")). \
                when(df2['race'] == lit("7"), lit("White-AI")). \
                when(df2['race'] == lit("8"), lit("White-Asian")). \
                when(df2['race'] == lit("9"), lit("White-HP")). \
                when(df2['race'] == lit("10"), lit("Black-AI")). \
                when(df2['race'] == lit("11"), lit("Black-Asian")). \
                when(df2['race'] == lit("12"), lit("Black-HP")). \
                when(df2['race'] == lit("13"), lit("AI-Asian")). \
                when(df2['race'] == lit("14"), lit("AI-HP")). \
                when(df2['race'] == lit("15"), lit("Asian-HP")). \
                when(df2['race'] == lit("16"), lit("W-B-AI")). \
                when(df2['race'] == lit("17"), lit("W-B-A")). \
                when(df2['race'] == lit("18"), lit("W-B-HP")). \
                when(df2['race'] == lit("19"), lit("W-AI-A")). \
                when(df2['race'] == lit("20"), lit("W-AI-HP")). \
                when(df2['race'] == lit("21"), lit("W-A-HP")). \
                when(df2['race'] == lit("22"), lit("B-AI-A")). \
                when(df2['race'] == lit("23"), lit("W-B-AI-A")). \
                when(df2['race'] == lit("24"), lit("W-AI-A-HP")). \
                when(df2['race'] == lit("25"), lit("Other 3 Race Combinations")). \
                when(df2['race'] == lit("26"), lit("Other 4 and 5 Race Combinations")). \
                 otherwise('UNDEFINED').alias('race'))


In [0]:
display(df3.head(10))


household_identifier,interview_time,final_survey_outcome,housing_unit_type,household_type,apartment/household_has_telephone,apartment/household_can_access_telephone_elsewhere,is_telephone_interview_acceptable,interview_type,family_income,geographical_division/location,race
000004795110719,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,TELEPHONE,"30,000 TO 34,999",EAST SOUTH CENTRAL,White Only
000004795110719,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,TELEPHONE,"30,000 TO 34,999",EAST SOUTH CENTRAL,White Only
000071691004941,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,PERSONAL,"40,000 TO 49,999",EAST SOUTH CENTRAL,White Only
000071691004941,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,PERSONAL,"40,000 TO 49,999",EAST SOUTH CENTRAL,White Only
000071691004941,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,PERSONAL,"40,000 TO 49,999",EAST SOUTH CENTRAL,White Only
000110177987986,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,PERSONAL,"75,000 TO 99,999",EAST SOUTH CENTRAL,Black Only
000110177987986,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF),YES,YES,YES,PERSONAL,"75,000 TO 99,999",EAST SOUTH CENTRAL,Black Only
000110206593381,2017/12,LANGUAGE BARRIER,"HOUSE, APARTMENT, FLAT",UNDEFINED,UNDEFINED,YES,UNDEFINED,PERSONAL,"LESS THAN $5,000",EAST SOUTH CENTRAL,UNDEFINED
000110284815680,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",CIVILIAN FEMALE PRIMARY INDIVIDUAL,YES,YES,YES,PERSONAL,"30,000 TO 34,999",EAST SOUTH CENTRAL,White Only
000110327856469,2017/12,CAPI COMPLETE,"HOUSE, APARTMENT, FLAT",CIVILIAN FEMALE PRIMARY INDIVIDUAL,YES,YES,YES,PERSONAL,"12,500 TO 14,999",EAST SOUTH CENTRAL,Black Only


##What is the count of responders per family income range (show top 10)?

In [0]:
df4 = df3.groupBy("family_income").count().orderBy(desc('count')).limit(10)

In [0]:
display(df4)

family_income,count
"LESS THAN $5,000",23527
"100,000 TO 149,999",17794
"75,000 TO 99,999",16557
"150,000 OR MORE",15704
"60,000 TO 74,999",13442
"50,000 TO 59,999",9971
"40,000 TO 49,999",9788
"30,000 TO 34,999",6743
"35,000 TO 39,999",6620
"20,000 TO 24,999",6312


##What is the count of responders per geographical division/location and race (show top 10)?

In [0]:
df5 = df3.groupBy("geographical_division/location").count().orderBy(desc('count')).limit(10)

In [0]:
display(df5)

geographical_division/location,count
SOUTH ATLANTIC,27699
PACIFIC,20890
MOUNTAIN,18500
WEST SOUTH CENTRAL,16537
EAST NORTH CENTRAL,15308
WEST NORTH CENTRAL,13068
MIDDLE ATLANTIC,12790
NEW ENGLAND,11296
EAST SOUTH CENTRAL,10368


##How many responders do not have telephone in their house, but can access a telephone elsewhere and telephone interview is accepted (show top 10)?

In [0]:
df6 = df3.select(df3.household_identifier).filter((df3["apartment/household_has_telephone"] == 'NO') &
                       (df3["apartment/household_can_access_telephone_elsewhere"] == 'YES') &
                       (df3["is_telephone_interview_acceptable"] == 'YES'))

In [0]:
display(df6.head(10))

household_identifier
180820609591100
198051896000201
031610601051005
100160530561100
281710004200257
281710004200257
281710004200257
281710004200257
281710004200257
281710004200257


In [0]:
print(str(df6.count()) + " responders do not have telephone in their house, but can access a telephone elsewhere and telephone interview is accepted")

634 responders do not have telephone in their house, but can access a telephone elsewhere and telephone interview is accepted

##How many responders can access to a telephone, but telephone interview is not accepted (show top 10)?

In [0]:
df7 = df3.select(df3.household_identifier).filter((df3["apartment/household_has_telephone"] == 'YES') &
                       (df3["is_telephone_interview_acceptable"] == 'NO'))

In [0]:
display(df7)

household_identifier


In [0]:
print(str(df7.count()) + " responders can access to a telephone, but telephone interview is not accepted")

0 responders can access to a telephone, but telephone interview is not accepted